# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [2]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [3]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [8]:
show(boston, allcols = true)

506×14 DataFrame
│ Row │ Crim    │ Zn      │ Indus   │ Chas  │ NOx     │ Rm      │ Age     │
│     │ Float64 │ Float64 │ Float64 │ Int64 │ Float64 │ Float64 │ Float64 │
├─────┼─────────┼─────────┼─────────┼───────┼─────────┼─────────┼─────────┤
│ 1   │ 0.00632 │ 18.0    │ 2.31    │ 0     │ 0.538   │ 6.575   │ 65.2    │
│ 2   │ 0.02731 │ 0.0     │ 7.07    │ 0     │ 0.469   │ 6.421   │ 78.9    │
│ 3   │ 0.02729 │ 0.0     │ 7.07    │ 0     │ 0.469   │ 7.185   │ 61.1    │
│ 4   │ 0.03237 │ 0.0     │ 2.18    │ 0     │ 0.458   │ 6.998   │ 45.8    │
│ 5   │ 0.06905 │ 0.0     │ 2.18    │ 0     │ 0.458   │ 7.147   │ 54.2    │
│ 6   │ 0.02985 │ 0.0     │ 2.18    │ 0     │ 0.458   │ 6.43    │ 58.7    │
│ 7   │ 0.08829 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 6.012   │ 66.6    │
│ 8   │ 0.14455 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 6.172   │ 96.1    │
│ 9   │ 0.21124 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 5.631   │ 100.0   │
│ 10  │ 0.17004 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 6.004   │ 85.9 

In [6]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);

In [7]:
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.09744,0.0,5.96,0,0.499,5.841,61.4,3.3779,5,279
2,0.34006,0.0,21.89,0,0.624,6.458,98.9,2.1185,4,437
3,0.06911,45.0,3.44,0,0.437,6.739,30.8,6.4798,5,398
4,2.37857,0.0,18.1,0,0.583,5.871,41.9,3.724,24,666
5,0.05644,40.0,6.41,1,0.447,6.758,32.9,4.0776,4,254
6,4.64689,0.0,18.1,0,0.614,6.98,67.6,2.5329,24,666
7,12.2472,0.0,18.1,0,0.584,5.837,59.7,1.9976,24,666
8,4.66883,0.0,18.1,0,0.713,5.976,87.9,2.5806,24,666
9,0.07978,40.0,6.41,0,0.447,6.482,32.1,4.1403,4,254


In [13]:
ols = GLM.lm(@formula(MedV ~ Rm + LStat + PTRatio + Tax + Crim + Age + Dis + Black + Rad + NOx + Chas + Indus + Zn), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

MedV ~ 1 + Rm + LStat + PTRatio + Tax + Crim + Age + Dis + Black + Rad + NOx + Chas + Indus + Zn

Coefficients:
──────────────────────────────────────────────────────────────────────────────────────
                 Estimate  Std. Error    t value  Pr(>|t|)      Lower 95%    Upper 95%
──────────────────────────────────────────────────────────────────────────────────────
(Intercept)   37.8241      5.73057      6.6004      <1e-9    26.5575       49.0906
Rm             3.75798     0.463919     8.10051     <1e-14    2.84589       4.67006
LStat         -0.540546    0.0547392   -9.87493     <1e-19   -0.648165     -0.432926
PTRatio       -0.916168    0.143259    -6.39519     <1e-9    -1.19782      -0.634514
Tax           -0.0134852   0.00438419  -3.07588     0.0022   -0.0221047    -0.00486569
Crim          -0.117502    0.0372

In [14]:
predict(ols, test)

101-element Array{Union{Missing, Float64},1}:
 22.50902113191417
 19.893216912697252
 30.28755369643735
 19.51752408216142
 37.69891594500925
 25.48469940760964
 17.10685812500771
 13.396328637646217
 30.5702893233068
 27.14352241767217
 28.538362150777473
  3.1815261624757056
 32.766106847008594
  ⋮
 26.968457455840753
 25.430442542259552
 23.21315265784553
  5.541277713840238
 15.892920309026962
 26.2258710070119
 24.139526451335332
 17.20531342979988
 37.098435596003696
 13.652899604486544
 38.83250388305304
 34.79553566906278

In [15]:
GLM.r2(ols)

0.7412019708279994

In [16]:
GLM.adjr2(ols)

0.7325974327736874